## 2015, 2016, and 2017 honey Colonies

#### For honey colonies 

To run the code successfully:
   - Need to have all files downloaded from: http://usda.mannlib.cornell.edu/MannUsda/viewDocumentInfo.do?documentID=1943
   - rename file name 'hcny_all_tables.csv' under 2018 tab to 'hcny_all_tables2018.csv'
   - rename file name 'hcny_all_tables.csv' under 2017 tab to 'hcny_all_tables2017.csv'
   - rename file name 'hcny_all_tables.csv' under 2016 tab to 'hcny_all_tables2016.csv'
   
Retrive the dataframes of each year from:
   - colonies_15
   - colonies_16
   - colonies_17

Retrive the dataframe for combined data:
   - honeycol_df

#### For honey 
To run the code successfully:
     - Need to have all files downloaded from: http://usda.mannlib.cornell.edu/MannUsda/viewDocumentInfo.do?documentID=1191
    - rename file name 'hony_all_tables.csv' under 2018 tab to 'hony_all_tables2018.csv'
        - released Mar 2018, contains honey data of year 2016 and 2017
    - rename file name 'hony_all_tables.csv' under 2017 tab to 'hony_all_tables2017.csv'
        - released Mar 2017, contains honey data of year 2015 and 2016
Retrive the dataframes of each year from:
   - honey_15
   - honey_16
   - honey_17

Retrive the dataframe for the combined:
   - honey_df

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import csv
import sklearn.preprocessing as prep
path = os.getcwd()

def get_honey(file_name):
    with open(path+'/'+file_name, encoding = "ISO-8859-1") as input_honey:
        honeyread = csv.reader(input_honey, delimiter = ',')
        honey = []
        header =[]
        for line in honeyread:
            if line[1] == 'd' and line[3] != '':
                #print(line)
                honey.append(line[2:])
            if line[1] == 'h': #and line[3] != '': 
                header.append(line[2:])

    # get the headers for each table
    i = 0
    count = 0
    title = []
    for line in header:
        if 'State' in line and line[2] != '':
            count += 1 
            title_j = [a + ' ' + b + ' ' + c for a,b,c in zip(header[i-1], header[i], header[i+1] )]
            line_clean =  [re.sub(r'\d+\W','',c) for c in title_j]
            title.append(line_clean[1:])
        i+=1


    ### put colonies data into df
    honey_df = pd.DataFrame(honey)

    honey_df = honey_df.iloc[:-18] # only take desire rows

    # remove the unwanted characters & numbers from the indexes 
    honey_df[0] = honey_df[0].str.replace('\d+\W','')
    honey_df[0] = honey_df[0].str.strip()

    df_list=[]
    for i in range(count-1): # the last table is unwanted, thus eliminate
        columns = [' '.join(col.split()).lower() for col in title[i]] # strip white spaces and make lower
        columns = columns[:-1]
        group_df = honey_df.iloc[:,1:len(columns)+1].groupby(honey_df.iloc[:,0]).nth(i)
        df = pd.DataFrame(group_df.values, columns = columns, index = group_df.index)
        df.index.name = None # remove unwanted index name

        # deal with the missing values 
        df = df.replace('-',0) # replace '-' with 0 (according to documentation)
        df = df.replace('(Z)', 0.5) # replace '(Z)' with 0.5 (according to documentation)
        df = df.drop(['United States'],axis = 0)
        df = df.apply(pd.to_numeric) # change the object type to int 

        # put df's into a list, numbers of dfs should equal to 'count'
        df_list.append(df)
    
    return df_list

In [ ]:
get_18_table = get_honey('hony_all_tables2018.csv')
get_17_table = get_honey('hony_all_tables2017.csv')

honey_15 = get_17_table[0]
honey_16 = get_18_table[0]
honey_17 = get_18_table[1]

honey_be = {'2015':honey_15, 
            '2016':honey_16,
            '2017':honey_17
           }
honey_df = pd.concat(honey_be)

In [ ]:
def get_colonies(file_name, year):
    # get path
    path = os.getcwd()
    
    # open the file
    with open(path+'/'+file_name, encoding = "ISO-8859-1") as input_honey:
        honeyread = csv.reader(input_honey, delimiter = ',')
        honey = []
        header =[]
        for line in honeyread:
            if line[1] == 'd' and line[3] != '':
                honey.append(line[2:])
            if line[1] == 'h': #and line[3] != '': 
                header.append(line[2:])

    # get the headers, and indexes
    i = 0
    count = 0
    index = []
    title = []
    for line in header:
        if 'State' in line:
            count += 1 
            if line[2] == '':
                index_i = [x for x in header[i-1]]
                # index_i = [str(year) + x for x in header[i-1]]
                index.append(index_i[1:])
                title_i = [a+' '+b for a,b in zip(header[i+1],header[i+2])]
                title_clean = [re.sub(r'\d+\W','',c) for c in title_i]
                title.append(title_clean[1:])
            if line[2] != '':
                title_j = [a + ' ' + b for a,b in zip(header[i-1], header[i] )]
                line_clean =  [re.sub(r'\d+\W','',c) for c in title_j]
                title.append(line_clean[1:])
        i+=1
    
    # put colonies data into df
    honey_df = pd.DataFrame(honey)
    
    # only interested state data
    if year != 2016:
        honey_df = honey_df.iloc[:-13]
    else:
        honey_df = honey_df.iloc[:-7]
    
    # remove the unwanted characters & numbers from the indexes 
    honey_df[0] = honey_df[0].str.replace('\d+\W','')
    honey_df[0] = honey_df[0].str.strip() # strip white space
     
    # create sepearte dataframes
    df_list=[]
    for i in range(count):
        columns = [' '.join(col.split()).lower() for col in title[i]] # strip white spaces and make lower
        group_df = honey_df.iloc[:,1:len(columns)+1].groupby(honey_df.iloc[:,0]).nth(i)
        df = pd.DataFrame(group_df.values, columns = columns, index = group_df.index)
        
        df.index.name = None # remove unwanted index name
        
        # deal with the missing values 
        df = df.replace('-',0) # replace '-' with 0 (according to documentation)
        df = df.replace('(Z)', 0.5) # replace '(Z)' with 0.5 (according to documentation)
        df = df.drop(['United States'],axis = 0)
        df = df.apply(pd.to_numeric) # change the object type to int 
        
        # update "Other States' index
        state_list = ['Other States','Connecticut','Maryland','Massachusetts','Oklahoma', 'New Mexico'] # a list of states to be sum
        new_other_state = df.loc[state_list,:].sum(axis = 0) # sum the columns for the list of states 
        df = df.drop(state_list,axis = 0) # drop the list of states
        df.loc['Other States']= new_other_state  # create a new index to store the sums

        # put the df's into a list, numbers of dfs should equal to 'count'
        df_list.append(df)
        

    # return(df_list, title, index,count,i) -> checked the number of dfs are correct
    return(df_list, title, index)

In [ ]:
# 2017 data
df_17, title, index= get_colonies('hcny_all_tables2018.csv',2018)
honey_17 = (df_17[0] + df_17[1] + df_17[2] + df_17[3])/4
disease_17 = (df_17[6] + df_17[7] + df_17[8] + df_17[9])/4
colonies_17 = pd.concat([honey_17, disease_17.iloc[:,:-1]], axis = 1)

# 2016
df_16, title, index= get_colonies('hcny_all_tables2017.csv',2017)
honey_16 = (df_16[0] + df_16[1] + df_16[2] + df_16[3])/4
disease_16 = (df_16[6] + df_16[7] + df_16[8] + df_16[9])/4
colonies_16 = pd.concat([honey_16, disease_16], axis = 1)


# 2015
df_15, title, index= get_colonies('hcny_all_tables2016.csv',2016)
honey_15 = (df_15[0] + df_15[1] + df_15[2] + df_15[3])/4
disease_15 = (df_15[6] + df_15[7] + df_15[8] + df_15[9])/4
colonies_15 = pd.concat([honey_15, disease_15.iloc[:,:-1]], axis = 1)


# concat
#honey = pd.concat([colonies_15,colonies_16,colonies_17], keys = ['2015','2016','2017'], sort = True)
honey_bee = {'2015':colonies_15, 
            '2016':colonies_16,
            '2017':colonies_17
           }
honeycol_df = pd.concat(honey_bee)

In [ ]:
concat_list = [honey_df, honeycol_df]
merged_df = pd.concat(concat_list, axis=1, sort = False)

In [ ]:
# check to see if merged correctly:
# merged_df.info()

In [ ]:
# drop repeated columns to prevent data leakage
del_col = ['honey producing colonies', 'production', 'number of colonies', 'maximum colonies']
clean_df = merged_df.drop(del_col, axis=1)

In [ ]:
# import weather data
weather = pd.read_csv(path+'/'+'weather_2015-2017.txt',index_col = [0,1])

weather.index.names = ('State','Year')

weather = weather.sort_index(level=1) 
# ensure both dataframes have consistent index types
weather.index = weather.index.set_levels([weather.index.levels[0].astype(str),weather.index.levels[1]]) 


merge_list = [clean_df, weather]
merged_df = pd.concat(merge_list, axis=1, sort = False)

# normalize every column except for the targeted varaible 
merged_df.iloc[:,1:] = prep.scale(merged_df.iloc[:,1:], axis = 0, with_mean = True, with_std = True)

# export to csv
path = os.getcwd() 
merged_df.to_csv('alldata.csv',sep = ',', encoding= 'utf-8')

merged_df.info()

# clean_df.join(weather, how = 'inner')
# error: merging with more than one level overlap on a multi-index is not implemented


# to retrive a row
# weather.loc[('Alabama', 2015)]

# weather.loc['Alabama']
#weather.loc['Iowa'].iloc[0]